# Les centres d'intérêts des députés influencent-ils leur participation aux séances en commission permanente ? (16/12/2018)


## Introduction

Introduction ici: contexte général



présenter la motivation

<b> Stratégie d'idenfication </b>
 présenter la stratégie d'idenfication
 
 <b> Algorithmes </b>
 
 Présenter l'aspect plus algorithmique 
 
 
A demander : 

Récupérer le champ lexical


préférences des députés 



jouer sur le nombre de catégories, voir le threshold de déclenchement 

idée: 

• définir des catégories
•gradient de préférences pour le député
•gradient de thématiques pour la séance
•voir les paires de thématiques qui reviennent le plus souvent les unes à la suite des autres pour en déduire une proximité entre les différentes thématiques 
    - idée probabilité d'avoir un mot à la suite d'un autre 
    
• calculer une distance thématique/préférence, voir le pouvoir prédictif de cette distance


première étape pour les mots: créer une boucle, 1 ligne numero du député, une autre avec toutes ses interventions à la suite (sans traitement) 
filtrer pour les séances en hémicycle uniquement

à partir de ça voir les mots qui sortent le plus souvent pour chacun des députés




 


In [10]:
from jyquickhelper import add_notebook_menu
add_notebook_menu()

## Créer un profil de préférences pour chaque député

Première chose : créer les catégories thématiques (regrouper les champs lexicaux). Agreger toutes les interventions de tous les députés / prendre uniquement les séances en hémicycle et agréger pour tous les députés

A la fin, avoir des mots qui permettent de classifier une thématique de séance
faire du train/test pour voir si ça marche bien


La première chose à faire est de récupérer un fichier contenant toutes les interventions de tous les députés. On importe le fichier <TT>words_mps.txt</TT> qui contient l'ensemble des interventions en hémicycle de l'ensemble des députés de la XIVème législature. Plus précisément, la colonne 0 correspond à l'id du député dans notre base de données (qui nous sera utile plus tard pour retrouver la présence des députés en commission ainsi que leur groupe parlementaire). La colonne 1 est la concaténation brute de toutes ses interventions que l'on va ensuite nettoyer pour extraire les centres d'intérêts de chacun des députés.

Nous utilisons un script autonome pour extraire les données à partir d'une base SQL (cf. annexes)

In [11]:
import pandas 
import numpy as np

#Attention à indiquer l'emplacement local du fichier words_mps.txt (accessible en local uniquement)

#df=pandas.read_csv('/Users/Gabriel/Desktop/words_mps.txt', sep='\t',header=None)
df = pandas.read_csv('/Users/Hugo/Documents/Cours/ENSAE/Python/python_deputes/words_mps.txt', sep='\t',header=None,na_filter=False)
df1 = df.values
df2 = np.array(df)
df.head()

,0,1
0,1,"('<p>Bla-bla-bla !</p>',)(""<p>Vous n'avez enco..."
1,2,"(""<p>Ce n'est pas fini !</p>"",)('<p>Lamentable..."
2,3,"(""<p>J'ai bien entendu la position du Gouverne..."
3,4,"(""<p>C'est vous qui le faites !</p>"",)(""<p>Mon..."
4,5,"(""<p>Monsieur le président, monsieur le minist..."


In [12]:
print(type(df))
print(type(df1))
print(type(df2))
print(df2.shape)
for i in range(0,648) :
    z = df2[i,1]
    z = z.replace("</p>","")
    z = z.replace('"<p>',"")
    z = z.replace("('<p>","")
    z = z.replace("',)(","")
    z = z.replace('",)(','')
    z = z.replace('",)','')
    z = z.replace("',)",'')
    z = z.replace("<p>",'')
    z = z.replace("'"," ")
    z = z.replace("!",'')
    z = z.replace(".",' ')
    df2[i,1] = z
print(df2)

<class 'pandas.core.frame.DataFrame'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
(648, 2)


Création de la base pour tokenisation avec une seule ligne avec tous les mots employés

In [97]:
import numpy as np

liste=""

for i in range(648):
    liste=liste+df1[i,1]

Nous procédons à la Tokénisation

[Le code pour une tokénisation sans enlever les stops words]
import nltk
#nltk.download('punkt')
from nltk.tokenize import word_tokenize
for i in range(0,648) :
    z = df2[i,1]
    z = ' - '.join(word_tokenize(z))
    df2[i,1] = z
    print(i)
print(df2)

In [14]:
# on enlève les stop-words et on tokénise. En somme on bigdate
import nltk
# nltk.download('stopwords') 
# [Gab il faut que tu télécharges ça, je le mets en commentaire car je l'ai déjà fait]
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
st = set(stopwords.words('french'))
for i in range(0,648) :
    z = df2[i,1]
    z = ' - '.join(w for w in word_tokenize(z) if w not in st)
    df2[i,1] = z

NameError: name 'df2' is not defined

Etapes suivantes 

• nettoyer le fichier pour ne garder que les mots clefs
• trouver un moyen de regrouper les mots clef par champ lexical et mettre des catégories dessus (apprentissage non supervisé ?)

Le code suivant permet de procéder à la lemmatization de notre base de données. Cela permettra d'extraire de manière optimale les stop words ainsi que compter comme les mêmes mots les verbes conjugués etc. 

In [3]:
from french_lefff_lemmatizer.french_lefff_lemmatizer import FrenchLefffLemmatizer

In [ ]:
import nltk
from nltk.stem import WordNetLemmatizer

In [14]:
print(df2)

[[1
  "Bla-bla-bla - Vous - encore - rien - dit - Vous - déjà - dit - Erreur - N - importe - quoi - Et - Afghanistan - C - pitoyable - C - sûr - De - les - réduire - Il - a - oublié - C - Ayrault - , - zéro - C - redressement - volatile - On - verra - ça - durée - Vous - absolument - répondu - problème - dette - Ben - voyons - Les - Américains - font - Tout - fait - Ce - week-end - encore - Comme - les - 35 - heures - En - effet - Il - a - sans - doute - bien - meilleur - Montebourg - , - homme - redressement - volatil - C - déjà - cas - Ah - non - Allons - , - monsieur - président - Quelle - valeur - ajoutée - voitures - sortent - chaînes - allemandes - ? - Je - voudrais - profiter - défense - cet - amendement - répondre - M - Eckert - , - a - comparé - taux - horaire - les - secteurs - automobiles - allemand - français - Je - prends - acte - chiffres - fournis - , - monsieur - rapporteur - général - On - peut - également - citer - marge - réalisée - les - constructeurs - , - peu - pr

## Attribuer une thématique aux séances en commission

Rebelotte mais pour les séances en commission, très similaire aux députés (cf. script autonome)

In [30]:
import pandas
import numpy as np

#Attention à indiquer l'emplacement local du fichier words_meetings.txt (accessible en local uniquement)

#df=pandas.read_csv('/Users/Gabriel/Desktop/words_meetings.txt', sep='\t',header=None)
df_ = pandas.read_csv('/Users/Hugo/Documents/Cours/ENSAE/Python/python_deputes/words_meetings.txt', sep='\t',header=None,na_filter=False)
# On s'occupe ici de se débarasser des interventions vides 
df_[2].replace('', np.nan, inplace=True)
df_.dropna(subset=[2], inplace=True)
df_1 = df_.values
df_2 = np.array(df_)
df_.head()

,0,1,2
0,3,145,J'invite les deux plus jeunes commissaires pré...
1,3,83,"Au nom du groupe socialiste, républicain et ci..."
2,3,145,Je constate qu'il n'y a qu'un seul candidat. I...
3,3,77,Nous en venons à l'élection des quatre vice-pr...
4,3,77,"Pour les postes de secrétaires, j'ai reçu les ..."


In [31]:
from french_lefff_lemmatizer.french_lefff_lemmatizer import FrenchLefffLemmatizer
lemmatizer = FrenchLefffLemmatizer()
lemmatizer.lemmatize("J'invite les deux plus jeunes commissaires",all)

("J'invite les deux plus jeunes commissaires", 'np')

In [25]:
print(type(df_))
print(type(df_1))
print(type(df_2))
print(df_2.shape)
for i in range (0,5839) :
    z = df_2[i,1]
    z1 = repr(z)
    z1 = z1.replace("</p>","")
    z1 = z1.replace('"<p>',"")
    z1 = z1.replace("('<p>","")
    z1 = z1.replace("',)(","")
    z1 = z1.replace('",)(','')
    z1 = z1.replace('",)','')
    z1 = z1.replace("',)",'')
    z1 = z1.replace("<p>",'')
    z1 = z1.replace("'"," ")
    z1 = z1.replace("!",'')
    z1 = z1.replace(".",' ')
    z1 = z1.replace("\\",'')
    df_2[i,1] = z1
print(df_2) 


<class 'pandas.core.frame.DataFrame'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
(5839, 2)
[[3
  ' (Commission du développement durable et de l aménagement du territoirePrésidence de M  Jacques Kossowski, président d âgeJ invite les deux plus jeunes commissaires présents, MM  Julien Aubert et Guillaume Larrivé, à venir siéger au bureau pour y remplir les fonctions de secrétaires d âge Le bureau d âge est constitué L ordre du jour appelle maintenant la nomination du bureau de la Commission, conformément à l article 39 du Règlement  Ce bureau comprend, outre le président, quatre vice-présidents et quatre secrétaires  Aux termes de l article 39 du Règlement, « la composition du bureau de chaque commission s efforce de reproduire la configuration politique de l Assemblée et d assurer la représentation de toutes ses composantes »  Les alinéas 4 à 6 du même article précisent que « les bureaux des Commissions sont élus au scrutin secret par catégorie de fonction  Lorsque, pour chaque cat

In [18]:
# on enlève les stop-words et on tokénise. En somme on bigdate
import nltk
# nltk.download('stopwords') 
# [Gab il faut que tu télécharges ça, je le mets en commentaire car je l'ai déjà fait]
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
stop_w=stopwords.words('french')
#stop_w=.append(stop_w=.append("ayant","grand-chose","après-midi", "au-delà", "devant", "octobre", "novembre", "action", "assemblée", "activité", "besoin", "définition", "adopté", "député", "spéciale", "spécial", "rappelle", "hong", "précédente", "information", "délai", "applicable", "expliqué", "propose", "relative", "indique", "vingt", "janvier", "puis", "jour", "lecture", "séance", "alinéa", "résolution", "adoption", "collègue", "cher", "collègues", "chers","bis", "  er", "rectifié", "question", "rédactionnel", "scrutin", "exposer", "identiques", "identique", "commission", "adopte", "rejette", "additionnel", "tendant", "examiné", "examine", "rejeté", "avis", "suivant", "estimé", "déclaré", "parce", "beaucoup", "afin", "madame", "sous", "bonne", "monsieur", "quelle", "quinze", "lors", "là", "long", "messieurs", "ici", "trois", "êtes", "serait", "seront", "dix", "mot", "vin", "mon", "hier", "date", "cinq", "celui", "allez", "après", "bureau", "cause", "sous-amendement", "certaine", "chose", "code", "compris", "général", "pris", "demande", "jeudi", "jamais", "juillet", "mois", "plusieurs", "mardi", "mercredi", "lundi", "quatre", "semaine", "suppression", "semble", "souvent", "vers", "jamais", "comité", "discussion", "liens", "lieux", "membres", "vendredi", "dernière", "donner", "délégation", "défendu", "défavorable", "exemple", "favorable", "fonction", "grand", "habitant", "haut", "juridique", "mars", "membre", "mettre", "mise", "ministère", "mission", "niveau", "oui", "organique", "objet", "notion", "norme", "pense", "première", "prendre", "principe", "procédure", "puisque", "rien", "sceaux", "réponse", "spécial", "vise", "vos","vote", "suppresion", "urgence", "rapporteur", "rapporteure", "décembre", "dans", "aujourd’hui", "j'ai", "l'on", "n'a", "qu'elle", "qu'il", "s'agit")
stop_liste=["ayant","grand-chose","après-midi", "au-delà", "devant", "octobre", "novembre", "action", "assemblée", "activité", "besoin", "définition", "adopté", "député", "spéciale", "spécial", 'rappelle', 'hong', "précédente", "information", "délai", 'applicable', 'expliqué', 'propose', 'relative', 'indique', 'vingt', 'janvier', 'puis', 'jour', 'lecture', 'séance', 'alinéa', 'résolution', 'adoption', 'collègue', 'cher', 'collègues', 'chers','bis', '  er', 'rectifié', 'question', 'rédactionnel', 'scrutin', 'exposer', 'identiques', 'identique', 'commission', 'adopte', 'rejette', 'additionnel', 'tendant', 'examiné', 'examine', 'rejeté', 'avis', 'suivant', 'estimé', 'déclaré', 'parce', 'beaucoup', 'afin', 'madame', 'sous', 'bonne', 'monsieur', 'quelle', 'quinze', 'lors', 'là', 'long', 'messieurs', 'ici', 'trois', 'êtes', 'serait', 'seront', 'dix', 'mot', 'vin', 'mon', 'hier', 'date', 'cinq', 'celui', 'allez', 'après', 'bureau', 'cause', 'sous-amendement', 'certaine', 'chose', 'code', 'compris', 'général', 'pris', 'demande', 'jeudi', 'jamais', 'juillet', 'mois', 'plusieurs', 'mardi', 'mercredi', 'lundi', 'quatre', 'semaine', 'suppression', 'semble', 'souvent', 'vers', 'jamais', 'comité', 'discussion', 'liens', 'lieux', 'membres', 'vendredi', 'dernière', 'donner', 'délégation', 'défendu', 'défavorable', 'exemple', 'favorable', 'fonction', 'grand', 'habitant', 'haut', 'juridique', 'mars', 'membre', 'mettre', 'mise', 'ministère', 'mission', 'niveau', 'oui', 'organique', 'objet', 'notion', 'norme', 'pense', 'première', 'prendre', 'principe', 'procédure', 'puisque', 'rien', 'sceaux', 'réponse', 'spécial', 'vise', 'vos', 'vote', 'suppresion', 'urgence', "rapporteur", "rapporteure", 'décembre', 'dans', "aujourd'hui", "j'ai", "l'on", "n'a", "qu'elle", "qu'il", "s'agit"]
for i in range(len(stop_liste)):
    stop_w.append(stop_liste[i])
st = set(stop_w)
for i in range(len(df_2)) :
    z = df_2[i,2]
    z = ' - '.join(w for w in word_tokenize(z) if w not in st)
    df_2[i,2] = z
print(df_2)

[[3 145
  "J'invite - les - deux - plus - jeunes - commissaires - présents - , - MM - . - Julien - Aubert - Guillaume - Larrivé - , - venir - siéger - bureau - remplir - les - fonctions - secrétaires - d'âge.Le - bureau - d'âge - constitué.L'ordre - jour - appelle - maintenant - nomination - bureau - Commission - , - conformément - l'article - 39 - Règlement - . - Ce - bureau - comprend - , - outre - président - , - quatre - vice-présidents - quatre - secrétaires - . - Aux - termes - l'article - 39 - Règlement - , - « - composition - bureau - chaque - commission - s'efforce - reproduire - configuration - politique - l'Assemblée - d'assurer - représentation - toutes - composantes - » - . - Les - alinéas - 4 - 6 - article - précisent - « - les - bureaux - Commissions - élus - scrutin - secret - catégorie - fonction - . - Lorsque - , - chaque - catégorie - fonction - , - nombre - candidats - n'est - supérieur - nombre - sièges - pourvoir - , - n'est - procédé - scrutin - . - Si - majorité

In [20]:
print(len(df_2))

160393


## Voir comment la proximité thématique prédit la participation

## Bibliographie


<span>&#8226;</span> Benjamin Monnery & Maxime Le Bihan, 2018. "Can Public and Private Sanctions Discipline Politicians? Evidence from the French Parliament," EconomiX Working Papers 2018-21, University of Paris Nanterre, EconomiX.

<span>&#8226;</span> Wu, A. H. (2017). Gender stereotyping in academia: Evidence from economics job market rumors forum.
